# Step 1: Import packages

In [ ]:
# Step 1 -- Import the packages you need: numpy, matplotlib.pyplot



# Step 2: Load data

In [ ]:
# Step 2 -- Load data

# Update your file names!
open_files = ['open_1.txt',...]
closed_files = ['closed_1.txt',...]

# Define column names
columns = ['time', 'recording']
sampling_freq = 400

open_data = np.array([])
closed_data = np.array([])

# Define a function to remove nan values
def nan_helper(y):
    return np.isnan(y), lambda z: z.nonzero()[0]

for file in open_files:
    this_open_data = np.genfromtxt(file, dtype=float, usecols=(0,1), skip_header=6, delimiter='\t', names=columns, encoding = 'unicode_escape')
    
    open_timestamps = this_open_data['time']
    open_recording = this_open_data['recording']
    nans, x = nan_helper(open_recording)
    open_recording[nans]= np.interp(x(nans), x(~nans), open_recording[~nans])

    np.concatenate(open_data,open_recording)

for file in closed_files:
    this_closed_data = np.genfromtxt(file, dtype=float, usecols=(0,1), skip_header=6, delimiter='\t', names=columns, encoding = 'unicode_escape')
    
    # Save recording & timestamps as variables
    closed_timestamps = this_closed_data['time']
    closed_recording = this_closed_data['recording']
    nans, x = nan_helper(closed_recording)
    closed_recording[nans]= np.interp(x(nans), x(~nans), closed_recording[~nans])

    np.concatenate(closed_data,closed_recording)


closed_recording

# Step 3: Plot raw data

In [ ]:
# Step 3A -- Plot one trial of open data

# Set up figure
fig,ax = plt.subplots(figsize=(10,4))

# Change the variables below if you'd like to plot eyes open instead
plt.plot(closed_timestamps,closed_recording)

# You may need to change the x label!
plt.xlabel('') 

# You may need to change the y label!
plt.ylabel('')

# You can uncomment the line below to restrict the x axis plotting -- for example, to zoom into alpha waves
#plt.xlim([45,50])

plt.show()

In [ ]:
# Step 3B -- Closed data plot



# Step 4: PSD

In [ ]:
from scipy import signal

# Define sliding window length (4 seconds, which will give us 2 full cycles at 0.5 Hz)
win = 4 * sampling_freq
freqs, psd = signal.welch(closed_recording, sampling_freq, nperseg=win)

# Plot the power spectrum
plt.figure(figsize=(8, 6))
plt.plot(freqs, psd)

# Uncomment the lines below to plot two lines
#open_freqs, open_psd = signal.welch(open_recording, sampling_freq, nperseg=win)
#plt.plot(open_freqs, open_psd)
#plt.legend(['Closed','Open'])

plt.xlabel('Frequency (Hz)')
plt.ylabel('Power spectral density (pV^2 / Hz)') # Check that these units make sense!
#plt.xlim([0,75]) # Change x limit

plt.title("Welch's PSD")
plt.show()

# Step 5: Spectrogram

In [ ]:
max = 20 # change the max value on your spectogram -- you may need to adjust this

wind = np.hanning(1024) # Create a "hanning" window with a given binning window size

# Create the spectrogram and plot
fig = plt.figure() 

# Change closed to open below to generate for open
f, tt, Sxx = signal.spectrogram(closed_recording,sampling_freq,wind,len(wind),len(wind)-1)

# You can change cmap & vmax if you want
plt.pcolormesh(tt,f,Sxx,cmap='viridis',vmax=vmax)
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (sec)')
# plt.ylim([0,75]) # set the ylimit
cbar = plt.colorbar()
cbar.ax.set_ylabel('Power ($V^2$)')
plt.show()